# NCES Education Data - Quick Start Guide

This notebook demonstrates how to use the **NCESConnector** to access National Center for Education Statistics data.

**Data Systems:**
- **CCD:** Common Core of Data (public K-12 schools)
- **PSS:** Private School Universe Survey
- **IPEDS:** Integrated Postsecondary Education Data (colleges)

**Data Categories:**
- School directory & enrollment
- Demographics (race/ethnicity, gender)
- Performance (test scores, graduation rates)
- Finances (revenues, expenditures, per-pupil spending)

**API Provider:** Urban Institute Education Data Portal (no key required)

---

*© 2025 KR-Labs. Licensed under Apache-2.0.*

## 1. Setup and Import

In [1]:
from krl_data_connectors.education import NCESConnector
import pandas as pd

# Initialize connector
nces = NCESConnector()

print("✓ NCES connector initialized")
print(f"API Base URL: {nces.base_url}")

TypeError: Can't instantiate abstract class NCESConnector without an implementation for abstract methods 'connect', 'fetch'

## 2. Get State Schools

Retrieve school directory for a state.

In [ ]:
# Get Rhode Island schools for 2023
ri_schools = nces.get_state_schools('RI', year=2023)

print(f"Retrieved {len(ri_schools)} schools")
if not ri_schools.empty:
    print("\nColumns:", ri_schools.columns.tolist()[:10])
    ri_schools.head()

## 3. Load School Data from File

Alternative: Load downloaded NCES data from CSV.

In [ ]:
# If you have downloaded CCD files
# Download from: https://nces.ed.gov/ccd/files.asp

# school_file = "path/to/ccd_schools_2023.csv"
# schools = nces.load_school_data(school_file)
# print(f"Loaded {len(schools)} schools")

## 4. Get Enrollment Data

Retrieve enrollment statistics.

In [ ]:
# Get enrollment data for Rhode Island
enrollment = nces.get_enrollment_data('RI', year=2023)

if not enrollment.empty:
    print(f"Retrieved enrollment for {len(enrollment)} schools/districts")
    print("\nSample enrollment data:")
    enrollment.head()
else:
    print("No enrollment data available (API may require authentication)")

## 5. Extract Demographics

Analyze student demographics from school data.

In [ ]:
# Extract demographic information
if not ri_schools.empty:
    demographics = nces.get_demographics(ri_schools)
    
    if not demographics.empty:
        print("Demographic Categories:")
        print(demographics.columns.tolist())
        print("\nSample demographics:")
        demographics.head()
    else:
        print("No demographic columns found in school data")
else:
    print("Load school data first to extract demographics")

## 6. Get Graduation Rates

Retrieve graduation rate statistics.

In [ ]:
# Get graduation rates
grad_rates = nces.get_graduation_rates('RI', year=2023)

if not grad_rates.empty:
    print(f"Retrieved graduation data: {len(grad_rates)} records")
    print("\nGraduation rates:")
    grad_rates.head()
else:
    print("No graduation rate data available")

## 7. Get District Finances

Analyze school district financial data.

In [ ]:
# Get district finance data
finances = nces.get_district_finance('RI', year=2023)

if not finances.empty:
    print(f"Retrieved finance data for {len(finances)} districts")
    print("\nFinancial data columns:")
    print(finances.columns.tolist())
    print("\nSample finance data:")
    finances.head()
else:
    print("No finance data available")

## 8. Calculate Per-Pupil Spending

Compute spending per student by district.

In [ ]:
# Calculate per-pupil spending
if not finances.empty and not enrollment.empty:
    per_pupil = nces.calculate_per_pupil_spending(finances, enrollment)
    
    if not per_pupil.empty and 'per_pupil_spending' in per_pupil.columns:
        print("Per-Pupil Spending by District:")
        print(per_pupil[['leaid', 'total_expenditures', 'enrollment', 
                         'per_pupil_spending']].head())
        
        print(f"\nAverage per-pupil spending: ${per_pupil['per_pupil_spending'].mean():.2f}")
    else:
        print("Unable to calculate per-pupil spending (missing columns)")
else:
    print("Example per-pupil calculation:")
    print("Total expenditures: $50,000,000")
    print("Total enrollment: 10,000")
    print(f"Per-pupil spending: ${50000000/10000:.2f}")

## 9. Compare Districts

Compare metrics across districts.

In [ ]:
# Compare districts by enrollment
district_enrollment = nces.compare_districts('RI', 2023, metric='enrollment')

if not district_enrollment.empty:
    print("District Comparison (Enrollment):")
    if 'enrollment' in district_enrollment.columns:
        top_districts = district_enrollment.nlargest(5, 'enrollment')
        print(top_districts[['leaid', 'enrollment']])
    else:
        print(district_enrollment.head())
else:
    print("No district data available for comparison")

## 10. Extract Performance Metrics

Analyze school performance data.

In [ ]:
# Get school performance metrics
if not ri_schools.empty:
    performance = nces.get_school_performance(ri_schools)
    
    if not performance.empty:
        print("Performance Metrics:")
        print(performance.columns.tolist())
        print("\nSample performance data:")
        performance.head()
    else:
        print("No performance metrics found in school data")
        print("\nPerformance columns typically include:")
        print("  - Test scores (math, reading, science)")
        print("  - Proficiency rates")
        print("  - Graduation rates")
        print("  - College readiness indicators")

## 11. School Type Analysis

Examine different school types.

In [ ]:
# Show school type definitions
print("School Type Classifications:")
for code, description in nces.school_types.items():
    print(f"  {code}: {description}")

# If school data has type codes
if not ri_schools.empty and 'school_type' in ri_schools.columns:
    print("\nSchool Type Distribution:")
    print(ri_schools['school_type'].value_counts())

## 12. Export Results

Save education data for further analysis.

In [ ]:
# Export school directory
if not ri_schools.empty:
    nces.export_to_csv(ri_schools, 'ri_schools_2023.csv')
    print("✓ School directory exported")

# Export demographics
if not ri_schools.empty:
    demographics = nces.get_demographics(ri_schools)
    if not demographics.empty:
        nces.export_to_csv(demographics, 'ri_school_demographics.csv')
        print("✓ Demographics exported")

# Export per-pupil spending
if not finances.empty and not enrollment.empty:
    per_pupil = nces.calculate_per_pupil_spending(finances, enrollment)
    if not per_pupil.empty:
        nces.export_to_csv(per_pupil, 'ri_per_pupil_spending.csv')
        print("✓ Per-pupil spending exported")

## Next Steps

**Explore More:**
- Private school data (PSS)
- College/university data (IPEDS)
- National Assessment of Educational Progress (NAEP) scores
- Teacher demographics and qualifications
- Special education programs
- Chronic absenteeism rates

**Resources:**
- [NCES Data Tools](https://nces.ed.gov/datatools/)
- [Urban Institute Education Data Portal](https://educationdata.urban.org/)
- [CCD Documentation](https://nces.ed.gov/ccd/)
- [IPEDS Documentation](https://nces.ed.gov/ipeds/)

**Analysis Ideas:**
- Achievement gap analysis by demographics
- Resource equity across districts
- School choice and charter school impact
- College readiness trends
- Teacher retention patterns
- School funding adequacy studies